In [2]:
# !pip install pymysql

In [61]:
import pymysql

In [62]:
con = pymysql.connect(
    host = 'localhost', 
    port = 3306, # int값
    user = 'root', 
    password = 'woody1234', 
    db = 'mldb3', 
    charset = 'utf8'
)

con # stream: 통로
print('1. 연결성공')

1. 연결성공


In [63]:
cur = con.cursor()
cur # 통로 안 객체 가리킴
print('2. 커서 객체 획득')

2. 커서 객체 획득


In [64]:
# sql = 'select * from member'
sql = """
    SELECT m.name, m.tel, b.content
    FROM board b
    INNER JOIN member m ON m.id = b.writer
    WHERE content LIKE "%java"
    """

In [65]:
result_count = cur.execute(sql) # 객체 row 갯수 가져옴

print('3. sql 전송 성공', result_count)

3. sql 전송 성공 2


In [66]:
rows = cur.fetchall()
print('4. 검색결과 ------------')
print(rows)

4. 검색결과 ------------
(('apple', '555555555', 'fun java'), ('apple', '555555555', 'good java'))


In [67]:
# 메모리가 잡혀있는 상태이므로 연결 끊어줘야 함
con.close()

In [68]:
for index, row in enumerate(rows):
    print(index, row)
#     print(index, row[0])

0 ('apple', '555555555', 'fun java')
1 ('apple', '555555555', 'good java')


In [69]:
# db 연동 함수
def connect():
    con = pymysql.connect(
        host = 'localhost', 
        port = 3306, # int값
        user = 'root', 
        password = 'woody1234', 
        db = 'mldb3', 
        charset = 'utf8'
    )
    print("1. 연결성공")

    cur = con.cursor()
    print("2. 커서 객체 획득")
    return cur, con

In [70]:
cur2, con = connect() 
cur2

1. 연결성공
2. 커서 객체 획득


In [71]:
def sql_execute(cur2, data):
    sql = """
    select o2.orderid, m.name, o2.productid, o2.totalprice
    from orderlist o2
    inner join `member` m
    on o2.memberid = m.id and m.id = %s
    order by productid desc
    """
    result = cur2.execute(sql, data)
    return result # db rows 개수 return

In [72]:
sql_result = sql_execute(cur2, 'park')
sql_result

5

In [73]:
## 1. 누가 어떤 물건을 얼마나 샀는가?

In [74]:
who = input("누가? >> ")

누가? >> park


In [75]:
sql_result2 = sql_execute(cur2, who)
sql_result2

5

In [76]:
def out_data(cur2):
    rows = cur2.fetchall()
    return rows

In [77]:
result_rows = out_data(cur2)
result_rows

((2, 'park', '101', 6000),
 (1, 'park', '100', 5000),
 (3, 'park', '100', 10000),
 (4, 'park', '100', 3000),
 (8, 'park', '100', 1000))

In [78]:
def db_close(con):
    con.close()

In [79]:
db_close(con)

In [80]:
## 제일 높은 금액은?

In [81]:
[x[3] for x in result_rows]

[6000, 5000, 10000, 3000, 1000]

In [82]:
max([x[3] for x in result_rows])

10000

In [83]:
## 제일 많이 산 물건의 id는?

In [84]:
import pandas as pd

In [85]:
result_df = pd.DataFrame(data=result_rows, columns=['주문id', 'name', '제품id', '주문총가격'])

In [86]:
result_df

,주문id,name,제품id,주문총가격
0,2,park,101,6000
1,1,park,100,5000
2,3,park,100,10000
3,4,park,100,3000
4,8,park,100,1000


In [87]:
[x for x in result_df['제품id']]

['101', '100', '100', '100', '100']

In [88]:
from collections import Counter

In [89]:
counter_id = Counter(result_df['제품id'])

In [90]:
counter_id

Counter({'101': 1, '100': 4})

In [91]:
max_id = counter_id.most_common(n=1)
print(max_id)
print(max_id[0][0])
max_id_result = max_id[0][0]
max_id_result

[('100', 4)]
100


'100'

In [92]:
def sql_execute2(cur2, data):
    sql = """
    select * from product
    where id = %s
    """
    result2 = cur2.execute(sql, data)
    return result2

In [93]:
cur2, con = connect()
cur2

1. 연결성공
2. 커서 객체 획득


In [94]:
sql_result2 = sql_execute2(cur2, max_id_result)
sql_result2

1

In [95]:
result_rows = out_data(cur2)
result_rows[0]

('100', 'java', 'fun java', '1000')

In [96]:
print("당신에게 추천하고 싶은 물건은 다음과 같습니다.")
print("-----------------------------------")
print("물건의 이름은 ", result_rows[0][1])
print("물건의 이름은 ", result_rows[0][2])
print("물건의 이름은 ", result_rows[0][3])

당신에게 추천하고 싶은 물건은 다음과 같습니다.
-----------------------------------
물건의 이름은  java
물건의 이름은  fun java
물건의 이름은  1000


In [97]:
def sql_execute3(con, cur2, data):
    sql = """
    insert into orderlist values (null, %s, %s, %s)
    """
    result2 = cur2.execute(sql, data)
    con.commit()
    return result2

In [98]:
order_question = input("구매하시겠습니까?(yes/no)")
if order_question == "yes":
    print("-------------구매를 진행합니다.-------------")
    your_id = input("당신의 id는 ")
    data_list = [your_id, result_rows[0][0], int(result_rows[0][3])]
    result3 = sql_execute3(con, cur2, data_list)
    if result3 == 1:
        print("----구매가 완료되었습니다.")
    else:
        print("----구매에 문제가 발생했습니다.") 

구매하시겠습니까?(yes/no)yes
-------------구매를 진행합니다.-------------
당신의 id는 park
----구매가 완료되었습니다.


In [1]:
# 추천을 하고자 하는 컨텐츠기반 룰을 정하고,
# 물건을 3개 이상 추천하는 추천시스템을 완성하시오.
# db의 데이터를 임의로 추가할 수 있음.
# 함수나 클래스로 모듈화해주세요.